Project: /mediapipe/_project.yaml
Book: /mediapipe/_book.yaml

<link rel="stylesheet" href="/mediapipe/site.css">

# Hand gesture recognition model customization guide

<table align="left" class="buttons">
  <td>
    <a href="https://colab.research.google.com/github/googlesamples/mediapipe/blob/main/examples/customization/gesture_recognizer.ipynb" target="_blank">
      <img src="https://developers.google.com/static/mediapipe/solutions/customization/colab-logo-32px_1920.png" alt="Colab logo"> Run in Colab
    </a>
  </td>

  <td>
    <a href="https://github.com/googlesamples/mediapipe/blob/main/examples/customization/gesture_recognizer.ipynb" target="_blank">
      <img src="https://developers.google.com/static/mediapipe/solutions/customization/github-logo-32px_1920.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

In [7]:
#@title License information
# Copyright 2023 The MediaPipe Authors.
# Licensed under the Apache License, Version 2.0 (the "License");
#
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

The MediaPipe Model Maker package is a low-code solution for customizing on-device machine learning (ML) Models.

This notebook shows the end-to-end process of customizing a gesture recognizer model for recognizing some common hand gestures in the [HaGRID](https://www.kaggle.com/datasets/innominate817/hagrid-sample-30k-384p) dataset.

## Prerequisites

Install the MediaPipe Model Maker package.

In [8]:
# !pip install --upgrade pip
# !pip install mediapipe-model-maker

Import the required libraries.

In [9]:
# from google.colab import files
import os
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import gesture_recognizer

import matplotlib.pyplot as plt

## Simple End-to-End Example

This end-to-end example uses Model Maker to customize a model for on-device gesture recognition.

### Get the dataset

The dataset for gesture recognition in model maker requires the following format: `<dataset_path>/<label_name>/<img_name>.*`. In addition, one of the label names (`label_names`) must be `none`. The `none` label represents any gesture that isn't classified as one of the other gestures.

This example uses a rock paper scissors dataset sample which is downloaded from GCS.

In [10]:
import os

import os
if not os.path.exists("rps_data_sample.zip"):
    !wget https://storage.googleapis.com/mediapipe-tasks/gesture_recognizer/rps_data_sample.zip
    !unzip rps_data_sample.zip
    print("File downloaded.")
else:
    print("File already exists.")


dataset_path = "rps_data_sample"

File already exists.


Verify the rock paper scissors dataset by printing the labels. There should be 4 gesture labels, with one of them being the `none` gesture.

In [11]:
print(dataset_path)
labels = []
for i in os.listdir(dataset_path):
  if os.path.isdir(os.path.join(dataset_path, i)):
    labels.append(i)
print(labels)

rps_data_sample
['none', 'scissors', 'rock', 'paper']


To better understand the dataset, plot a couple of example images for each gesture.

In [12]:
NUM_EXAMPLES = 5

for label in labels:
  label_dir = os.path.join(dataset_path, label)
  example_filenames = os.listdir(label_dir)[:NUM_EXAMPLES]
  fig, axs = plt.subplots(1, NUM_EXAMPLES, figsize=(10,2))
  for i in range(NUM_EXAMPLES):
    axs[i].imshow(plt.imread(os.path.join(label_dir, example_filenames[i])))
    axs[i].get_xaxis().set_visible(False)
    axs[i].get_yaxis().set_visible(False)
  fig.suptitle(f'Showing {NUM_EXAMPLES} examples for {label}')

plt.show()

/tmp/ipykernel_6490/544766458.py:13: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### Run the example
The workflow consists of 4 steps which have been separated into their own code blocks.

**Load the dataset**

Load the dataset located at `dataset_path` by using the `Dataset.from_folder` method. When loading the dataset, run the pre-packaged hand detection model from MediaPipe Hands to detect the hand landmarks from the images. Any images without detected hands are ommitted from the dataset. The resulting dataset will contain the extracted hand landmark positions from each image, rather than images themselves.

The `HandDataPreprocessingParams` class contains two configurable options for the data loading process:
* `shuffle`: A boolean controlling whether to shuffle the dataset. Defaults to true.
* `min_detection_confidence`: A float between 0 and 1 controlling the confidence threshold for hand detection.

Split the dataset: 80% for training, 10% for validation, and 10% for testing.

In [13]:
data = gesture_recognizer.Dataset.from_folder(
    dirname=dataset_path,
    hparams=gesture_recognizer.HandDataPreprocessingParams()
)
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

INFO:tensorflow:Loading image /home/bankab/AGH_ISS/Semestr_1/ICK/second_test/rps_data_sample/paper/258.jpg
INFO:tensorflow:Loading image /home/bankab/AGH_ISS/Semestr_1/ICK/second_test/rps_data_sample/none/1501.jpg


libEGL warning: egl: failed to create dri2 screen
libEGL warning: egl: failed to create dri2 screen
I0000 00:00:1743154425.243208    6490 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1743154425.274214    6997 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.0.2-arch1.2), renderer: llvmpipe (LLVM 19.1.7, 256 bits)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1743154425.352678    7001 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743154425.393713    7005 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743154425.505570    7007 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


INFO:tensorflow:Loading image /home/bankab/AGH_ISS/Semestr_1/ICK/second_test/rps_data_sample/rock/218.jpg
INFO:tensorflow:Loading image /home/bankab/AGH_ISS/Semestr_1/ICK/second_test/rps_data_sample/none/190.jpg
INFO:tensorflow:Loading image /home/bankab/AGH_ISS/Semestr_1/ICK/second_test/rps_data_sample/scissors/188.jpg
INFO:tensorflow:Loading image /home/bankab/AGH_ISS/Semestr_1/ICK/second_test/rps_data_sample/rock/182.jpg
INFO:tensorflow:Loading image /home/bankab/AGH_ISS/Semestr_1/ICK/second_test/rps_data_sample/none/950.jpg
INFO:tensorflow:Loading image /home/bankab/AGH_ISS/Semestr_1/ICK/second_test/rps_data_sample/paper/176.jpg
INFO:tensorflow:Loading image /home/bankab/AGH_ISS/Semestr_1/ICK/second_test/rps_data_sample/rock/431.jpg
INFO:tensorflow:Loading image /home/bankab/AGH_ISS/Semestr_1/ICK/second_test/rps_data_sample/none/1107.jpg
INFO:tensorflow:Loading image /home/bankab/AGH_ISS/Semestr_1/ICK/second_test/rps_data_sample/none/142.jpg
INFO:tensorflow:Loading image /home/bank

2025-03-28 10:34:25.836900: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-28 10:34:25.860778: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-28 10:34:25.864592: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

INFO:tensorflow:Load valid hands with size: 473, num_label: 4, labels: none,paper,rock,scissors.


INFO:tensorflow:Load valid hands with size: 473, num_label: 4, labels: none,paper,rock,scissors.


**Train the model**

Train the custom gesture recognizer by using the create method and passing in the training data, validation data, model options, and hyperparameters. For more information on model options and hyperparameters, see the [Hyperparameters](#hyperparameters) section below.

In [14]:
hparams = gesture_recognizer.HParams(export_dir="exported_model")
options = gesture_recognizer.GestureRecognizerOptions(hparams=hparams)
model = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hand_embedding (InputLayer  [(None, 128)]             0         
 )                                                               
                                                                 
 batch_normalization (Batch  (None, 128)               512       
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 128)               0         
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 custom_gesture_recognizer_  (None, 4)                 516       
 out (Dense)                                                     
                                                             

INFO:tensorflow:Training the models...
2025-03-28 10:34:29.822640: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 1/10


2025-03-28 10:34:34.385450: I external/local_xla/xla/service/service.cc:168] XLA service 0x798a7564b020 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-03-28 10:34:34.385511: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 Ti, Compute Capability 7.5
2025-03-28 10:34:34.401953: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-03-28 10:34:34.451642: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1743154474.689913    7041 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


    186/Unknown - 6s 7ms/step - loss: 0.8300 - categorical_accuracy: 0.3683

2025-03-28 10:34:36.924120: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 4704911066168351405
2025-03-28 10:34:36.924157: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 5483793822605503007
2025-03-28 10:34:36.924182: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 17446564653262796110


189/189 [==============================] - 8s 15ms/step - loss: 0.8329 - categorical_accuracy: 0.3651 - val_loss: 0.7090 - val_categorical_accuracy: 0.5319 - lr: 0.0010


2025-03-28 10:34:38.184017: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 2182011353960004307
2025-03-28 10:34:38.184056: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 16946087962080311901


Epoch 2/10
189/189 [==============================] - 2s 9ms/step - loss: 0.6215 - categorical_accuracy: 0.4921 - val_loss: 0.5286 - val_categorical_accuracy: 0.6383 - lr: 9.9000e-04
Epoch 3/10
189/189 [==============================] - 2s 8ms/step - loss: 0.5406 - categorical_accuracy: 0.5873 - val_loss: 0.4368 - val_categorical_accuracy: 0.6383 - lr: 9.8010e-04
Epoch 4/10
189/189 [==============================] - 2s 9ms/step - loss: 0.5024 - categorical_accuracy: 0.6217 - val_loss: 0.3763 - val_categorical_accuracy: 0.6809 - lr: 9.7030e-04
Epoch 5/10
189/189 [==============================] - 2s 8ms/step - loss: 0.4980 - categorical_accuracy: 0.6455 - val_loss: 0.3410 - val_categorical_accuracy: 0.7234 - lr: 9.6060e-04
Epoch 6/10
189/189 [==============================] - 2s 8ms/step - loss: 0.4795 - categorical_accuracy: 0.6640 - val_loss: 0.3163 - val_categorical_accuracy: 0.7447 - lr: 9.5099e-04
Epoch 7/10
189/189 [==============================] - 1s 8ms/step - loss: 0.4637 - ca

**Evaluate the model performance**

After training the model, evaluate it on a test dataset and print the loss and accuracy metrics.

In [15]:
loss, acc = model.evaluate(test_data, batch_size=1)
print(f"Test loss:{loss}, Test accuracy:{acc}")

48/48 [==============================] - 1s 2ms/step - loss: 0.4871 - categorical_accuracy: 0.7083
Test loss:0.4870522916316986, Test accuracy:0.7083333134651184


2025-03-28 10:34:54.322763: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 2182011353960004307
2025-03-28 10:34:54.322787: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 16946087962080311901
2025-03-28 10:34:54.322799: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 11530370678210044963


**Export to Tensorflow Lite Model**

After creating the model, convert and export it to a Tensorflow Lite model format for later use on an on-device application. The export also includes model metadata, which includes the label file.

In [16]:
model.export_model()
!ls exported_model

Using existing files at /tmp/model_maker/gesture_recognizer/palm_detection_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/hand_landmark_full.tflite
INFO:tensorflow:Assets written to: /tmp/tmp_uq53mab/saved_model/assets


INFO:tensorflow:Assets written to: /tmp/tmp_uq53mab/saved_model/assets
2025-03-28 10:34:56.348067: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2025-03-28 10:34:56.348087: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.


best_model_weights.data-00000-of-00001	epoch_models		 metadata.json
best_model_weights.index		gesture_recognizer.task
checkpoint				logs


2025-03-28 10:34:56.348324: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp_uq53mab/saved_model
2025-03-28 10:34:56.349112: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2025-03-28 10:34:56.349123: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp_uq53mab/saved_model
2025-03-28 10:34:56.351194: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2025-03-28 10:34:56.351733: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2025-03-28 10:34:56.368633: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmp_uq53mab/saved_model
2025-03-28 10:34:56.375227: I tensorflow/cc/saved_model/loader.cc:316] SavedModel load for tags { serve }; Status: success: OK. Took 26903 microseconds.
Summary on the non-converted ops:
---------------------------------
 * Accep

## Run the model on-device

To use the TFLite model for on-device usage through MediaPipe Tasks, refer to the Gesture Recognizer [overview page](https://developers.google.com/mediapipe/solutions/vision/gesture_recognizer).

## Hyperparameters {:#hyperparameters}


You can further customize the model using the `GestureRecognizerOptions` class, which has two optional parameters for `ModelOptions` and `HParams`. Use the `ModelOptions` class to customize parameters related to the model itself, and the `HParams` class to customize other parameters related to training and saving the model.

`ModelOptions` has one customizable parameter that affects accuracy:
* `dropout_rate`: The fraction of the input units to drop. Used in dropout layer. Defaults to 0.05.
* `layer_widths`: A list of hidden layer widths for the gesture model. Each element in the list will create a new hidden layer with the specified width. The hidden layers are separated with BatchNorm, Dropout, and ReLU. Defaults to an empty list(no hidden layers).

`HParams` has the following list of customizable parameters which affect model accuracy:
* `learning_rate`: The learning rate to use for gradient descent training. Defaults to 0.001.
* `batch_size`: Batch size for training. Defaults to 2.
* `epochs`: Number of training iterations over the dataset. Defaults to 10.
* `steps_per_epoch`: An optional integer that indicates the number of training steps per epoch. If not set, the training pipeline calculates the default steps per epoch as the training dataset size divided by batch size.
* `shuffle`: True if the dataset is shuffled before training. Defaults to False.
* `lr_decay`: Learning rate decay to use for gradient descent training. Defaults to 0.99.
* `gamma`: Gamma parameter for focal loss. Defaults to 2

Additional `HParams` parameter that does not affect model accuracy:
* `export_dir`: The location of the model checkpoint files and exported model files.

For example, the following trains a new model with the dropout_rate of 0.2 and learning rate of 0.003.

In [18]:
hparams = gesture_recognizer.HParams(learning_rate=0.003, export_dir="exported_model_2")
model_options = gesture_recognizer.ModelOptions(dropout_rate=0.2)
options = gesture_recognizer.GestureRecognizerOptions(model_options=model_options, hparams=hparams)
model_2 = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hand_embedding (InputLayer  [(None, 128)]             0         
 )                                                               
                                                                 
 batch_normalization_1 (Bat  (None, 128)               512       
 chNormalization)                                                
                                                                 
 re_lu_1 (ReLU)              (None, 128)               0         
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 custom_gesture_recognizer_  (None, 4)                 516       
 out (Dense)                                                     
                                                           

INFO:tensorflow:Training the models...


Epoch 1/10
    186/Unknown - 2s 5ms/step - loss: 0.8473 - categorical_accuracy: 0.4140

2025-03-28 10:35:27.904092: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 4704911066168351405
2025-03-28 10:35:27.904133: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 5483793822605503007
2025-03-28 10:35:27.904166: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 17446564653262796110


189/189 [==============================] - 3s 10ms/step - loss: 0.8445 - categorical_accuracy: 0.4153 - val_loss: 0.3494 - val_categorical_accuracy: 0.7660 - lr: 0.0030
Epoch 2/10


2025-03-28 10:35:28.910687: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 2182011353960004307
2025-03-28 10:35:28.910726: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 16946087962080311901
2025-03-28 10:35:28.910737: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 11530370678210044963


189/189 [==============================] - 1s 8ms/step - loss: 0.5777 - categorical_accuracy: 0.5794 - val_loss: 0.2593 - val_categorical_accuracy: 0.8298 - lr: 0.0030
Epoch 3/10
189/189 [==============================] - 2s 8ms/step - loss: 0.5644 - categorical_accuracy: 0.6058 - val_loss: 0.1956 - val_categorical_accuracy: 0.8511 - lr: 0.0029
Epoch 4/10
189/189 [==============================] - 1s 8ms/step - loss: 0.4927 - categorical_accuracy: 0.6587 - val_loss: 0.1891 - val_categorical_accuracy: 0.8298 - lr: 0.0029
Epoch 5/10
189/189 [==============================] - 1s 8ms/step - loss: 0.5062 - categorical_accuracy: 0.6534 - val_loss: 0.1990 - val_categorical_accuracy: 0.8298 - lr: 0.0029
Epoch 6/10
189/189 [==============================] - 1s 8ms/step - loss: 0.4813 - categorical_accuracy: 0.6402 - val_loss: 0.1744 - val_categorical_accuracy: 0.8723 - lr: 0.0029
Epoch 7/10
189/189 [==============================] - 1s 8ms/step - loss: 0.4809 - categorical_accuracy: 0.6640 - va

Evaluate the newly trained model.

In [19]:
loss, accuracy = model_2.evaluate(test_data)
print(f"Test loss:{loss}, Test accuracy:{accuracy}")

2/2 [==============================] - 1s 15ms/step - loss: 0.4960 - categorical_accuracy: 0.7500
Test loss:0.49600863456726074, Test accuracy:0.75


2025-03-28 10:35:50.997216: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 2182011353960004307
2025-03-28 10:35:50.997238: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 16946087962080311901
2025-03-28 10:35:50.997251: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 11530370678210044963
